In [109]:
import pandas as pd
import numpy as np
import random
import csv
import os
import math
from tqdm import tqdm

traindf = pd.read_pickle("./pickles/df_small_train.pkl")
testdf = pd.read_pickle("./pickles/df_small_test.pkl")
trydf = pd.read_pickle("./pickles/df_small_train_test_yk.pkl")

In [108]:
def random_solution(df):
    
    csv_name = "./results/" + csv_name
    
    # get list of hotels per search id
    results_per_search = df.groupby('srch_id').size()
    
    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    all_ranks = []
    
    for i, ideetje in enumerate(ids):
        df_temp = df[df["srch_id"] == ideetje]
        
        ranks = df_temp["position"].values
        
        # shuffle ranks
        random.shuffle(ranks)
        all_ranks.extend(ranks)

    df["predicted_rank"] = all_ranks
    return df

In [3]:
def price_quality(df):

    df['pricequality'] = df['price_usd'] / df['prop_starrating']
    
    return df

In [111]:
def price_quality_solution(df):
    
    column_values = df["srch_id"].values
    ids =  pd.unique(column_values) 

    total_ids = len(ids)    
    
    all_propids = []    
    
    for i in tqdm(range(len(ids))):
        ideetje = ids[i]
        
        # get all rows that belong with this search
        df_temp = df[df["srch_id"] == ideetje]
        
        # get list of property ids
        prop_ids = df_temp["prop_id"].values
        
        # rank the property ids based on price/quality
        prop_ids_sorted = [x for _, x in sorted(zip(list(df_temp["pricequality"]), prop_ids))] 
        
        all_propids.extend(prop_ids_sorted)
    
    df["predicted_rank"] = all_propids
    return df
            

In [112]:
testdf = price_quality(testdf)
price_quality_solution(df=testdf)

100%|████████████████████████████████████████████████████████████████████████████| 1996/1996 [00:01<00:00, 1416.16it/s]


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,pricequality,predicted_rank
2165493,145778,2013-04-06 06:42:59,5,219,NaN,NaN,219,269,2,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.000000,55739
2165494,145778,2013-04-06 06:42:59,5,219,NaN,NaN,219,6133,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.333333,92214
2165495,145778,2013-04-06 06:42:59,5,219,NaN,NaN,219,6982,4,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,73.500000,113918
2165496,145778,2013-04-06 06:42:59,5,219,NaN,NaN,219,19854,3,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.333333,30977
2165497,145778,2013-04-06 06:42:59,5,219,NaN,NaN,219,21157,4,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.750000,19854
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3050502,205312,2013-01-21 00:25:32,15,55,NaN,NaN,55,128786,4,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.575000,107311
3050503,205312,2013-01-21 00:25:32,15,55,NaN,NaN,55,130224,4,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.767500,117039
3050504,205312,2013-01-21 00:25:32,15,55,NaN,NaN,55,133084,4,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.787500,1769
3050505,205312,2013-01-21 00:25:32,15,55,NaN,NaN,55,135061,3,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29.023333,103834


In [6]:
def change_time(df):
    
    if 'date_time' in df.columns:
        # Change time information to year and month columns
        df["date_time"] = pd.to_datetime(df["date_time"])
        df["year"] = df["date_time"].dt.year
        df["month"] = df["date_time"].dt.month

        #Delete column date-time
        df = df.drop('date_time', 1)
        
    return df

In [7]:
X = ["0", "1", "3", "20", "8"]
Y = [0, 1, 3, 20, 8]


In [8]:
srtd = [x for _,x in sorted(zip(Y,X))]
print(srtd)

['0', '1', '3', '8', '20']


100%|████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:01<00:00, 1665.23it/s]


In [10]:
display(predicted_df)

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,pricequality,propid_predicted
3303361,221792,2012-11-19 13:36:31,5,219,NaN,NaN,219,1230,3,3.5,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,96.333333,23268
3303362,221792,2012-11-19 13:36:31,5,219,NaN,NaN,219,1546,2,4.0,...,NaN,NaN,1.0,0.0,13.0,0,NaN,0,72.500000,21155
3303363,221792,2012-11-19 13:36:31,5,219,NaN,NaN,219,3572,2,4.0,...,NaN,NaN,0.0,0.0,NaN,0,NaN,0,69.500000,14536
3303364,221792,2012-11-19 13:36:31,5,219,NaN,NaN,219,6623,3,4.5,...,NaN,NaN,-1.0,0.0,15.0,0,NaN,0,76.333333,25075
3303365,221792,2012-11-19 13:36:31,5,219,NaN,NaN,219,8586,3,3.5,...,NaN,NaN,0.0,0.0,15.0,0,NaN,0,73.000000,110738
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45163,3003,2013-01-24 13:54:01,5,219,NaN,NaN,219,32625,4,4.5,...,NaN,NaN,NaN,NaN,NaN,1,996.56,1,55.000000,12248
45164,3003,2013-01-24 13:54:01,5,219,NaN,NaN,219,44693,4,4.5,...,NaN,NaN,1.0,0.0,24.0,0,NaN,0,65.750000,44693
45165,3003,2013-01-24 13:54:01,5,219,NaN,NaN,219,70568,5,4.5,...,NaN,NaN,0.0,0.0,73.0,0,NaN,0,64.000000,32243
45166,3003,2013-01-24 13:54:01,5,219,NaN,NaN,219,103937,5,4.5,...,NaN,NaN,-1.0,0.0,18.0,0,NaN,0,63.800000,29286


In [11]:
from sklearn.metrics import ndcg_score

# test
true_relevance = np.asarray([[10, 0, 0, 1, 5]])
scores = np.asarray([[.1, .2, .3, 4, 70]])
ndcg_score(true_relevance, scores)

0.6956940443813076

In [12]:
# score of predicted properties
true_propids = predicted_df["prop_id"]
predicted_propids = predicted_df["propid_predicted"]

In [125]:
def calculate_score(df):

    column_values = df["srch_id"].values
    
    ids =  pd.unique(column_values) 
    
    total_ids = len(ids)    

    total_score = 0
    
    for i in tqdm(range(len(ids))):
        ideetje = ids[i]
        
        # get all rows that belong with this search
        df_temp = df[df["srch_id"] == ideetje]
        
        # get the actual and predicted ranks
        actual_rank = list(df_temp["position"])
        predicted_rank = list(df_temp["predicted_rank"])
        
        
#         predicted_propids = list(df_temp["propid_predicted"])
#         propids = list(df_temp["prop_id"])
        
    
        if i == 0:
#             print("propids: ")
#             print(propids)
            
            print("ranks:")
            print(actual_rank)
            
#             print("sorted propids")
#             print(sorted_propids)
            
            print("predicted rank")
            print(predicted_rank)
            
            #         ranks = [predicted_propids.index(propid) for propid in propids]
            
#         predicted_propids = np.asarray([predicted_propids])
#         true_propids = np.asarray([sorted_propids])
        actual_rank = np.asarray([actual_rank])
        predicted_rank = np.asarray([predicted_rank])
        
        score = ndcg_score(actual_rank, predicted_rank)
#         print(score)
        total_score += score
        
    # score is averaged over all querys
    return total_score/total_ids

In [127]:
trydf = pd.read_pickle("./pickles/df_small_train_test_yk.pkl")
predicted_random = random_solution(df=trydf, csv_name="nothing", write_csv=False, add_column=True)
calculate_score(predicted_random)

  7%|█████▌                                                                        | 144/1998 [00:00<00:02, 712.20it/s]

ranks:
[7, 31, 32, 21, 34, 18, 1, 8, 19, 27, 3, 17, 12, 15, 14, 26, 20, 6, 11, 4, 33, 28, 25, 30, 2, 9, 35, 23, 29, 13, 24]
predicted rank
[14, 1, 26, 24, 20, 35, 23, 21, 18, 19, 25, 29, 32, 33, 34, 11, 12, 28, 31, 13, 9, 17, 3, 15, 27, 2, 30, 6, 7, 4, 8]


100%|█████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:02<00:00, 748.97it/s]


0.8290433206740115

In [114]:
# this should be one
trydf = pd.read_pickle("./pickles/df_small_train_test_yk.pkl")
trydf["predicted_rank"] = trydf["position"]
calculate_score(trydf)

 10%|███████▍                                                                      | 191/1998 [00:00<00:02, 860.81it/s]

ranks:
[7, 31, 32, 21, 34, 18, 1, 8, 19, 27, 3, 17, 12, 15, 14, 26, 20, 6, 11, 4, 33, 28, 25, 30, 2, 9, 35, 23, 29, 13, 24]
predicted rank
[7, 31, 32, 21, 34, 18, 1, 8, 19, 27, 3, 17, 12, 15, 14, 26, 20, 6, 11, 4, 33, 28, 25, 30, 2, 9, 35, 23, 29, 13, 24]


100%|████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:01<00:00, 1040.95it/s]


1.0

In [17]:
propids = [5, 8, 6, 3]
predicted_propids = [3, 8, 5, 6]
ranks = [predicted_propids.index(propid) for propid in propids]
print(ranks)

[2, 1, 3, 0]


In [18]:
def dcg_from_ranking(y_true, ranking):
    """Discounted cumulative gain (DCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    DCG @k : float
    """
    y_true = np.asarray(y_true)
    ranking = np.asarray(ranking)
    rel = y_true[ranking]
    gains = 2 ** rel - 1
    discounts = np.log2(np.arange(len(ranking)) + 2)
    return np.sum(gains / discounts)


def ndcg_from_ranking(y_true, ranking):
    """Normalized discounted cumulative gain (NDCG) at rank k
    Parameters
    ----------
    y_true : array-like, shape = [n_samples]
        Ground truth (true relevance labels).
    ranking : array-like, shape = [k]
        Document indices, i.e.,
            ranking[0] is the index of top-ranked document,
            ranking[1] is the index of second-ranked document,
            ...
    k : int
        Rank.
    Returns
    -------
    NDCG @k : float
    """
    k = len(ranking)
    best_ranking = np.argsort(y_true)[::-1]
    best = dcg_from_ranking(y_true, best_ranking[:k])
    return dcg_from_ranking(y_true, ranking) / best

In [48]:
def test_score(df):

    column_values = df["srch_id"].values
    ids =  pd.unique(column_values) 
    
    total_ids = len(ids)    

    total_score = 0
    
    for i in tqdm(range(len(ids))):
        ideetje = ids[i]
        
        # get all rows that belong with this search
        df_temp = df[df["srch_id"] == ideetje]
        
        predicted_propids = list(df_temp["propid_predicted"])
        propids = list(df_temp["prop_id"])
        
        ranks = [predicted_propids.index(propid) for propid in propids]
        
#         true_propids = np.asarray([propids])
#         ranks = np.asarray([ranks])
        
        score = ndcg_from_ranking(true_propids, ranks)
        total_score += score
        
    # score is averaged over all querys
    return total_score/total_ids

In [49]:
assert ndcg_from_ranking([5, 3, 2], [0, 1, 2]) == 1.0

In [50]:
test_score(test)

100%|█████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:08<00:00, 233.55it/s]


1.0

In [51]:
test_score(predicted_random)

100%|█████████████████████████████████████████████████████████████████████████████| 1998/1998 [00:08<00:00, 235.87it/s]


1.0